In [1]:
!pip install flask-ngrok

In [2]:
!pip install -U jsonpickle

In [ ]:
from flask import Flask, request, Response
from flask_ngrok import run_with_ngrok
import jsonpickle
import numpy as np
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import datetime



# Initialize the Flask application
app = Flask(__name__)
run_with_ngrok(app) 

FACE_DETECTOR_PATH = '/content/drive/My Drive/Reconocimiento De Emociones/src/haarcascade_frontalface_default.xml'
RUTA_DE_DATA_GUARDADA = 'content/drive/My Drive/Reconocimiento De Emociones/data/'
# Create the model
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))

# route http posts to this method
@app.route('/api/test', methods=['POST'])
def test():
    r = request
    # convert string of image data to uint8
    nparr = np.fromstring(r.data, np.uint8)
    # decode image
    img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
    # do some fancy processing here....
    # build a response dict to send back to client
    response = {'message': 'image received. size={}x{}'.format(img.shape[1],
                                                               img.shape[0])}
    # encode response using jsonpickle
    response_pickled = jsonpickle.encode(response)

    return Response(response=response_pickled, status=200, 
                    mimetype="application/json")
    
@app.route("/predecir", methods=["POST"])
def predecir():
    r = request
    # convert string of image data to uint8
    nparr = np.fromstring(r.data, np.uint8)
    # decode image
    image = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    detector = cv2.CascadeClassifier('/content/drive/My Drive/Reconocimiento De Emociones/src/haarcascade_frontalface_default.xml')
    rects = detector.detectMultiScale(image, scaleFactor=1.1, minNeighbors=5,minSize=(30, 30),
                                      flags = cv2.CASCADE_SCALE_IMAGE)
    # construct a list of bounding boxes from the detection
    rects = [(int(x), int(y), int(x + w), int(y + h)) for (x, y, w, h) in rects]
		# update the data dictionary with the faces detected
    response = {"num_faces": len(rects), "faces": rects, "success": True }
    response_pickled = jsonpickle.encode(response)
    return Response(response=response_pickled, status=200, 
                    mimetype="application/json")

@app.route("/emociones", methods=["POST"])
def detectarEmociones():
    model.load_weights('/content/drive/My Drive/Reconocimiento De Emociones/src/model.h5')

    # prevents openCL usage and unnecessary logging messages
    cv2.ocl.setUseOpenCL(False)

    # Diccionario de emociones
    emotion_dict = {0: "Enojado", 1: "Disgustado", 2: "Temerosa", 3: "Feliz", 4: "Neutral", 5: "Triste", 6: "Sorprendido"}
    #Hora actual
    hora_actual = datetime.datetime.now()
    # start the webcam feed
    #cap = cv2.VideoCapture(0)
    r = request
    # convert string of image data to uint8
    nparr = np.fromstring(r.data, np.uint8)
    # decode image
    image = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
    #codigo de guardado
    #imagennueva = cv2.imread(image)
    #Guardamos la imagen
    #cv2.imwrite('kokonete.jpg',image)
    fecha= "%s%s%s%s%s%srostrosindetectar.jpg" % ( hora_actual.year ,
                                                   hora_actual.month ,
                                                   hora_actual.day , 
                                                   hora_actual.hour ,
                                                   hora_actual.minute ,
                                                   hora_actual.second)
    RUTA_FINAL = '/content/drive/My Drive/Reconocimiento De Emociones/data/' + fecha
    cv2.imwrite(RUTA_FINAL,image)
    #--fin de codigo de guardado
    facecasc = cv2.CascadeClassifier(FACE_DETECTOR_PATH)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = facecasc.detectMultiScale(gray,scaleFactor=1.3, minNeighbors=5)
    rects = facecasc.detectMultiScale(image, scaleFactor=1.1, minNeighbors=5,
		                      	minSize=(30, 30), flags = cv2.CASCADE_SCALE_IMAGE)
    auxFrame = image.copy()
    for (x, y, w, h) in faces:
        rostro = auxFrame[y:y+h,x:x+w]
        rostro = cv2.resize(rostro,(150,150),interpolation= cv2.INTER_CUBIC)
        cv2.rectangle(image, (x, y-50), (x+w, y+h+10), (255, 0, 0), 2)
        roi_gray = gray[y:y + h, x:x + w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray, (48, 48)), -1), 0)
        prediction = model.predict(cropped_img)
        maxindex = int(np.argmax(prediction))
        cv2.putText(image, emotion_dict[maxindex], (x+10, y-5), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        #codigo de guardado
        #imagennueva = cv2.imread(image)
        fecha= "%s%s%s%s%s%srostrodetectado.jpg" % ( hora_actual.year ,
                                                     hora_actual.month ,
                                                     hora_actual.day , 
                                                     hora_actual.hour , 
                                                     hora_actual.minute ,
                                                     hora_actual.second)
        RUTA_FINAL = '/content/drive/My Drive/Reconocimiento De Emociones/data/' + fecha
        cv2.imwrite(RUTA_FINAL,image)
        #--fin de codigo de guardado
    rects = [(int(x), int(y), int(x + w), int(y + h)) for (x, y, w, h) in rects]
    response = {"num_faces": len(faces), "emocion" :emotion_dict[maxindex] , "faces": rects, "success": True  , "nombre_imagen": fecha , "URL_DRIVE":RUTA_FINAL}
    response_pickled = jsonpickle.encode(response)
    return Response(response=response_pickled, status=200, 
                mimetype="application/json")

def _grab_image(path=None, stream=None, url=None):
	# if the path is not None, then load the image from disk
	if path is not None:
		image = cv2.imread(path)
	# otherwise, the image does not reside on disk
	else:	
		# if the URL is not None, then download the image
		if url is not None:
			resp = urllib.request.urlopen(url)
			data = resp.read()
		# if the stream is not None, then the image has been uploaded
		elif stream is not None:
			data = stream.read()
		# convert the image to a NumPy array and then read it into
		# OpenCV format
		image = np.asarray(bytearray(data), dtype="uint8")
		image = cv2.imdecode(image, cv2.IMREAD_COLOR)
	# return the image
	return image




# start flask app
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://f5fd960d93a7.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
